# 베이지안

In [ ]:
# 사전확률
A = 1/2
not_A = 1/2

strawberry_for_A = 7/10
strawberry_for_B = 5/10

# 사후확률
posterior = (strawberry_for_A * A) / ((strawberry_for_A * A) + (strawberry_for_B * not_A))

A, round(posterior, 2)

# 부트스트래핑

In [ ]:
# 200 samples bootstrapping
sample_200 = []

for _ in range(10000):
  sample200 = np.random.choice(final['Income'], 200, replace=True)
  sample_200.append(sample200.mean())